In [1]:
import json
import pandas as pd
import time
import numpy as np
import os, json, glob


In [2]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', -1)

/var/folders/xp/rqvwstmn67n30hhthb72sjbw0000gp/T/ipykernel_16906/648210843.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#### Residential sales - scraped from iproperty

In [3]:
iproperty = pd.read_csv('scraped_data.csv')
iproperty.shape

(2037, 13)

In [4]:
iproperty.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3
0,johor,sale,1250000,1250000,"Taman Setia Eco Garden, Taman Setia Eco Garden, Johor",1.502719,103.598537,5440.0,3200,SQUARE_FEET,Johor,Gelang Patah,NaN
1,johor,sale,935000,935000,"Jalan Setia Tropika, Taman Setia Tropika, Prescint 10, Johor",1.545577,103.717554,2240.0,2420,SQUARE_FEET,Johor,Setia Tropika,NaN
2,johor,sale,3150000,3150000,"LEISURE FARM, JOHOR, Johor",1.420036,103.630483,6070.0,4120,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN
3,johor,sale,1096960,588000,"Taman Impian Emas, Skudai, Johor",1.570172,103.670159,NaN,From 1868,NaN,Johor,Skudai,Iconia Garden Residence @ Taman Impian Emas
4,johor,sale,1600000,1600000,"ECO SPRING, EKOFLORA, MOUNT AUSTIN, ECO SPRING, EKOFLORA, MOUNT AUSTIN, Johor",1.588081,103.759622,2800.0,3120,SQUARE_FEET,Johor,Johor Bahru,NaN


In [5]:
iproperty.groupby('state')['address'].nunique().reset_index()

,state,address
0,johor,1678


In [6]:
iproperty.state.unique()

array(['johor'], dtype=object)

In [7]:
## Taking unique list of property from scraped data
iproperty = iproperty.drop_duplicates(subset=['address',
                                            'lat','lng',
                                            'min_price','max_price',
                                             'builtUp'], keep='first')

In [8]:
iproperty.shape

(1991, 13)

In [9]:
iproperty.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3
0,johor,sale,1250000,1250000,"Taman Setia Eco Garden, Taman Setia Eco Garden, Johor",1.502719,103.598537,5440.0,3200,SQUARE_FEET,Johor,Gelang Patah,NaN
1,johor,sale,935000,935000,"Jalan Setia Tropika, Taman Setia Tropika, Prescint 10, Johor",1.545577,103.717554,2240.0,2420,SQUARE_FEET,Johor,Setia Tropika,NaN
2,johor,sale,3150000,3150000,"LEISURE FARM, JOHOR, Johor",1.420036,103.630483,6070.0,4120,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN
3,johor,sale,1096960,588000,"Taman Impian Emas, Skudai, Johor",1.570172,103.670159,NaN,From 1868,NaN,Johor,Skudai,Iconia Garden Residence @ Taman Impian Emas
4,johor,sale,1600000,1600000,"ECO SPRING, EKOFLORA, MOUNT AUSTIN, ECO SPRING, EKOFLORA, MOUNT AUSTIN, Johor",1.588081,103.759622,2800.0,3120,SQUARE_FEET,Johor,Johor Bahru,NaN


In [10]:
## Calculating the mean price for each property
iproperty["price_mean"] = np.where(
    iproperty["max_price"] != 0, 
    (iproperty["min_price"] + iproperty["max_price"])/2, 
    iproperty["min_price"]
)

In [11]:
# Capped property price at 5mil
iproperty = iproperty[iproperty['price_mean']<=5000000]

In [12]:
iproperty.price_mean.max()

4675000.0

In [13]:
# Principal amount estimate
iproperty['principal_amt'] = iproperty['price_mean']*0.9

In [14]:
# Monthly payment estimate
iproperty['monthly_pmt'] = iproperty['principal_amt']*((0.045/12)*((1+0.045/12)**360))/(((1+0.045/12)**360)-1)

In [15]:
iproperty.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt
0,johor,sale,1250000,1250000,"Taman Setia Eco Garden, Taman Setia Eco Garden, Johor",1.502719,103.598537,5440.0,3200,SQUARE_FEET,Johor,Gelang Patah,NaN,1250000.0,1125000.0,5700.209736
1,johor,sale,935000,935000,"Jalan Setia Tropika, Taman Setia Tropika, Prescint 10, Johor",1.545577,103.717554,2240.0,2420,SQUARE_FEET,Johor,Setia Tropika,NaN,935000.0,841500.0,4263.756882
2,johor,sale,3150000,3150000,"LEISURE FARM, JOHOR, Johor",1.420036,103.630483,6070.0,4120,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN,3150000.0,2835000.0,14364.528534
3,johor,sale,1096960,588000,"Taman Impian Emas, Skudai, Johor",1.570172,103.670159,NaN,From 1868,NaN,Johor,Skudai,Iconia Garden Residence @ Taman Impian Emas,842480.0,758232.0,3841.850158
4,johor,sale,1600000,1600000,"ECO SPRING, EKOFLORA, MOUNT AUSTIN, ECO SPRING, EKOFLORA, MOUNT AUSTIN, Johor",1.588081,103.759622,2800.0,3120,SQUARE_FEET,Johor,Johor Bahru,NaN,1600000.0,1440000.0,7296.268461


In [16]:
# Income estimate
iproperty['income'] = iproperty['monthly_pmt']/0.35

In [17]:
# Creating income group based on income estimate
conditions = [
    (iproperty['income'] <= 4849),
    (iproperty['income'] >= 4850) & (iproperty['income'] <= 10959),
    (iproperty['income'] >= 10960) & (iproperty['income'] <= 15039),
    (iproperty['income'] > 15040)
    ]

# create a list of the values we want to assign for each condition
values = ['B40', 'M40','T20_T1','T20_T2']

# create a new column and use np.select to assign values to it using our lists as arguments
iproperty['income_group'] = np.select(conditions, values)

# display updated DataFrame
iproperty.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group
0,johor,sale,1250000,1250000,"Taman Setia Eco Garden, Taman Setia Eco Garden, Johor",1.502719,103.598537,5440.0,3200,SQUARE_FEET,Johor,Gelang Patah,NaN,1250000.0,1125000.0,5700.209736,16286.313530,T20_T2
1,johor,sale,935000,935000,"Jalan Setia Tropika, Taman Setia Tropika, Prescint 10, Johor",1.545577,103.717554,2240.0,2420,SQUARE_FEET,Johor,Setia Tropika,NaN,935000.0,841500.0,4263.756882,12182.162521,T20_T1
2,johor,sale,3150000,3150000,"LEISURE FARM, JOHOR, Johor",1.420036,103.630483,6070.0,4120,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN,3150000.0,2835000.0,14364.528534,41041.510096,T20_T2
3,johor,sale,1096960,588000,"Taman Impian Emas, Skudai, Johor",1.570172,103.670159,NaN,From 1868,NaN,Johor,Skudai,Iconia Garden Residence @ Taman Impian Emas,842480.0,758232.0,3841.850158,10976.714738,T20_T1
4,johor,sale,1600000,1600000,"ECO SPRING, EKOFLORA, MOUNT AUSTIN, ECO SPRING, EKOFLORA, MOUNT AUSTIN, Johor",1.588081,103.759622,2800.0,3120,SQUARE_FEET,Johor,Johor Bahru,NaN,1600000.0,1440000.0,7296.268461,20846.481319,T20_T2


In [18]:
# Encode lat lon to geohash with 2 precision:
## Precision 7: 153m x 153m
## Precision 6: 1.22km x 0.61km
import pygeohash as gh
iproperty['GEOHASH7'] = iproperty.apply(lambda x: gh.encode(x.lat, x.lng, precision=7), axis=1)
iproperty['GEOHASH6'] = iproperty.apply(lambda x: gh.encode(x.lat, x.lng, precision=6), axis=1)

In [19]:
iproperty.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group,GEOHASH7,GEOHASH6
0,johor,sale,1250000,1250000,"Taman Setia Eco Garden, Taman Setia Eco Garden, Johor",1.502719,103.598537,5440.0,3200,SQUARE_FEET,Johor,Gelang Patah,NaN,1250000.0,1125000.0,5700.209736,16286.313530,T20_T2,w238t3w,w238t3
1,johor,sale,935000,935000,"Jalan Setia Tropika, Taman Setia Tropika, Prescint 10, Johor",1.545577,103.717554,2240.0,2420,SQUARE_FEET,Johor,Setia Tropika,NaN,935000.0,841500.0,4263.756882,12182.162521,T20_T1,w23bb1k,w23bb1
2,johor,sale,3150000,3150000,"LEISURE FARM, JOHOR, Johor",1.420036,103.630483,6070.0,4120,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN,3150000.0,2835000.0,14364.528534,41041.510096,T20_T2,w238n4t,w238n4
3,johor,sale,1096960,588000,"Taman Impian Emas, Skudai, Johor",1.570172,103.670159,NaN,From 1868,NaN,Johor,Skudai,Iconia Garden Residence @ Taman Impian Emas,842480.0,758232.0,3841.850158,10976.714738,T20_T1,w238zjf,w238zj
4,johor,sale,1600000,1600000,"ECO SPRING, EKOFLORA, MOUNT AUSTIN, ECO SPRING, EKOFLORA, MOUNT AUSTIN, Johor",1.588081,103.759622,2800.0,3120,SQUARE_FEET,Johor,Johor Bahru,NaN,1600000.0,1440000.0,7296.268461,20846.481319,T20_T2,w23c115,w23c11


#### User address data
    - User address are mapped to property data within 153m x 153m grid (geohash precision 7)
    - Precision 7 is more accurate but will only cover small number of users

In [59]:
## Read sample address data
df_address = pd.read_csv('Johor_Address.csv')
print(df_address.shape)
df_address.head()

(45, 4)


,user_id,user_address,latitude,longitude
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524
1,1002,"JALAN ILHAM 19,TAMAN ILHAM,KLUANG,86000,JOHOR",2.049248,103.343521
2,1003,"JALAN KENARI 11,TAMAN SCIENTEX HELICONIA,PASIR GUDANG,81700,JOHOR",1.517881,103.914729
3,1004,"JALAN SRI PERKASA 2/6,TAMAN TAMPOI UTAMA,JOHOR BAHRU,81200,JOHOR",1.505086,103.686263
4,1005,"JALAN NIBONG 1,TAMAN KOTA KULAI,KULAI,81000,JOHOR",1.666773,103.577618


In [60]:
# Encode lat lon to geohash with precision 9:
## Precision 9: 4.77m x 4.77m
import pygeohash as gh
df_address['GEOHASH'] = df_address.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=9), axis=1)
df_address.head()

,user_id,user_address,latitude,longitude,GEOHASH
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv
1,1002,"JALAN ILHAM 19,TAMAN ILHAM,KLUANG,86000,JOHOR",2.049248,103.343521,w237xth3w
2,1003,"JALAN KENARI 11,TAMAN SCIENTEX HELICONIA,PASIR GUDANG,81700,JOHOR",1.517881,103.914729,w23bssk6k
3,1004,"JALAN SRI PERKASA 2/6,TAMAN TAMPOI UTAMA,JOHOR BAHRU,81200,JOHOR",1.505086,103.686263,w238x3ypc
4,1005,"JALAN NIBONG 1,TAMAN KOTA KULAI,KULAI,81000,JOHOR",1.666773,103.577618,w239kzqvt


In [61]:
## Get first 7 character from geohash to get the grid of size 153m x 153m 
df_address['GEOHASH7'] = df_address['GEOHASH'].str[:7]

In [62]:
df_address.head()

,user_id,user_address,latitude,longitude,GEOHASH,GEOHASH7
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn
1,1002,"JALAN ILHAM 19,TAMAN ILHAM,KLUANG,86000,JOHOR",2.049248,103.343521,w237xth3w,w237xth
2,1003,"JALAN KENARI 11,TAMAN SCIENTEX HELICONIA,PASIR GUDANG,81700,JOHOR",1.517881,103.914729,w23bssk6k,w23bssk
3,1004,"JALAN SRI PERKASA 2/6,TAMAN TAMPOI UTAMA,JOHOR BAHRU,81200,JOHOR",1.505086,103.686263,w238x3ypc,w238x3y
4,1005,"JALAN NIBONG 1,TAMAN KOTA KULAI,KULAI,81000,JOHOR",1.666773,103.577618,w239kzqvt,w239kzq


In [63]:
mapped_df = pd.merge(df_address,iproperty,on='GEOHASH7',how='inner')

In [64]:
mapped_df.head()

,user_id,user_address,latitude,longitude,GEOHASH,GEOHASH7,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group,GEOHASH6
0,1006,"JALAN SIERRA PERDANA 2/11,TAMAN SIERRA PERDANA,MASAI,81750,JOHOR",1.479644,103.871048,w23b7tked,w23b7tk,johor,sale,458000,458000,"Jalan Sierra Perdana 2, Taman Sierra perdana, 81750, Johor",1.480105,103.870840,1170,1170,SQUARE_FEET,Johor,Masai,NaN,458000.0,412200.0,2088.556847,5967.305277,M40,w23b7t
1,1006,"JALAN SIERRA PERDANA 2/11,TAMAN SIERRA PERDANA,MASAI,81750,JOHOR",1.479644,103.871048,w23b7tked,w23b7tk,johor,sale,468000,468000,"jalan sierra Perdana 2/, Taman Sierra Perdana, 81750, Johor",1.480105,103.870840,1170,1800,SQUARE_FEET,Johor,Masai,NaN,468000.0,421200.0,2134.158525,6097.595786,M40,w23b7t
2,1016,"PANGSAPURI SRI AKASIA JALAN PERSISIRAN,TAMAN TAMPOI INDAH,JOHOR,81200,JOHOR",1.510382,103.683963,w238x6uqe,w238x6u,johor,sale,285000,285000,"Jalan Titiwangsa Utama, Taman Tampoi Indah, 81200, Johor",1.510436,103.683952,NaN,1028,SQUARE_FEET,Johor,Tampoi,Sri Akasia,285000.0,256500.0,1299.647820,3713.279485,B40,w238x6
3,1017,"JALAN SENTRAL 23/A,TAMAN NUSA SENTRAL,NUSAJAYA,79100,JOHOR",1.470665,103.624369,w238q58zn,w238q58,johor,sale,670000,670000,"Jalan Sentral 23, Taman Nusa Sentral, 79100, Johor",1.470665,103.624369,1540,2270,SQUARE_FEET,Johor,Iskandar Puteri (Nusajaya),NaN,670000.0,603000.0,3055.312418,8729.464052,M40,w238q5
4,1020,"PRIMA REGENCY, JALAN MASAI BARU,MASAI,81750,JOHOR",1.523900,103.812704,w23bdm6j7,w23bdm6,johor,sale,330000,330000,"Jalan Masai Baru, Johor Bahru, 81100, Johor",1.523912,103.812702,NaN,1050,SQUARE_FEET,Johor,Johor Bahru,Prima Regency,330000.0,297000.0,1504.855370,4299.586772,B40,w23bdm


In [65]:
mapped_df.shape

(11, 25)

In [66]:
income_grouped = mapped_df.groupby('income_group')['user_id'].count().reset_index()
income_grouped.head()

,income_group,user_id
0,B40,7
1,M40,4


#### User address data
    - User address are mapped to property data within  1.22km × 0.61km grid (geohash precision 6)
    - Precision 6 is less accurate but covers more user

In [67]:
df_address.head()

,user_id,user_address,latitude,longitude,GEOHASH,GEOHASH7
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn
1,1002,"JALAN ILHAM 19,TAMAN ILHAM,KLUANG,86000,JOHOR",2.049248,103.343521,w237xth3w,w237xth
2,1003,"JALAN KENARI 11,TAMAN SCIENTEX HELICONIA,PASIR GUDANG,81700,JOHOR",1.517881,103.914729,w23bssk6k,w23bssk
3,1004,"JALAN SRI PERKASA 2/6,TAMAN TAMPOI UTAMA,JOHOR BAHRU,81200,JOHOR",1.505086,103.686263,w238x3ypc,w238x3y
4,1005,"JALAN NIBONG 1,TAMAN KOTA KULAI,KULAI,81000,JOHOR",1.666773,103.577618,w239kzqvt,w239kzq


In [68]:
## Get first 7 character from geohash to get the grid of size 153m x 153m 
df_address['GEOHASH6'] = df_address['GEOHASH'].str[:6]

In [69]:
df_address.head()

,user_id,user_address,latitude,longitude,GEOHASH,GEOHASH7,GEOHASH6
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn,w238wk
1,1002,"JALAN ILHAM 19,TAMAN ILHAM,KLUANG,86000,JOHOR",2.049248,103.343521,w237xth3w,w237xth,w237xt
2,1003,"JALAN KENARI 11,TAMAN SCIENTEX HELICONIA,PASIR GUDANG,81700,JOHOR",1.517881,103.914729,w23bssk6k,w23bssk,w23bss
3,1004,"JALAN SRI PERKASA 2/6,TAMAN TAMPOI UTAMA,JOHOR BAHRU,81200,JOHOR",1.505086,103.686263,w238x3ypc,w238x3y,w238x3
4,1005,"JALAN NIBONG 1,TAMAN KOTA KULAI,KULAI,81000,JOHOR",1.666773,103.577618,w239kzqvt,w239kzq,w239kz


In [70]:
mapped_df2 = pd.merge(df_address,iproperty,on='GEOHASH6',how='inner')

In [71]:
mapped_df2.head()

,user_id,user_address,latitude,longitude,GEOHASH,GEOHASH7_x,GEOHASH6,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3,price_mean,principal_amt,monthly_pmt,income,income_group,GEOHASH7_y
0,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn,w238wk,johor,sale,710000,710000,"Jalan utama , Taman mutiara Rini, 81200, Johor",1.519905,103.638693,3238,1820,SQUARE_FEET,Johor,Skudai,NaN,710000.0,639000.0,3237.719130,9250.626085,M40,w238wke
1,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn,w238wk,johor,sale,950000,950000,"Jalan Mutiara 2/, Opal Mutiara Mas Scientex Skudai Cluster Corner , 81300, Johor",1.518402,103.634706,2813,3837,SQUARE_FEET,Johor,Skudai,NaN,950000.0,855000.0,4332.159399,12377.598283,T20_T1,w238wk2
2,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn,w238wk,johor,sale,648000,648000,"Jalan Utama, Taman Mutiara Rini, Johor",1.519905,103.638693,NaN,2240,SQUARE_FEET,Johor,Skudai,NaN,648000.0,583200.0,2954.988727,8442.824934,M40,w238wke
3,1001,"JALAN JASA 18,TAMAN MUTIARA RINI,JOHOR BAHRU,81300,JOHOR",1.516775,103.643524,w238wkngv,w238wkn,w238wk,johor,sale,579000,579000,"Jalan utama, Taman Mutiara Rini, 81300, Johor",1.519905,103.638693,1750,2200,SQUARE_FEET,Johor,Skudai,NaN,579000.0,521100.0,2640.337150,7543.820427,M40,w238wke
4,1006,"JALAN SIERRA PERDANA 2/11,TAMAN SIERRA PERDANA,MASAI,81750,JOHOR",1.479644,103.871048,w23b7tked,w23b7tk,w23b7t,johor,sale,0,367000,"Iskandar Malaysia, Masai, Johor",1.481903,103.868333,NaN,From 969,NaN,Johor,Masai,Meridin Bayvue,367000.0,330300.0,1673.581578,4781.661652,B40,w23b7tf


In [72]:
mapped_df2.shape

(114, 26)

In [73]:
income_grouped2 = mapped_df2.groupby('income_group')['user_id'].count().reset_index()
income_grouped2.head()

,income_group,user_id
0,B40,35
1,M40,56
2,T20_T1,8
3,T20_T2,15


#### Note:
- By using geohash precision 7, user will be mapped more accurately as the size of grid is smaller. However, only small number of users can be mapped:\
    - B40: 7 users
    - M40: 4 users
- Tried using geohash precision 6 and more users can be mapped and tagged the income group. Need to note that the trade-off is the accuracy as this method covers all users within appx. 1km grid from a particular property